# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096870


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/33 [00:00<?, ?it/s]

Rendering models:   6%|▌         | 2/33 [00:05<01:24,  2.72s/it]

Rendering models:   9%|▉         | 3/33 [00:05<00:59,  1.98s/it]

Rendering models:  12%|█▏        | 4/33 [00:05<00:41,  1.44s/it]

Rendering models:  15%|█▌        | 5/33 [00:07<00:43,  1.57s/it]

...

Rendering models:  88%|████████▊ | 29/33 [00:11<00:00,  5.42it/s]

Rendering models:  91%|█████████ | 30/33 [00:12<00:00,  4.97it/s]

Rendering models:  97%|█████████▋| 32/33 [00:12<00:00,  4.71it/s]

Rendering models: 100%|██████████| 33/33 [00:12<00:00,  5.60it/s]

not-logged-in-025360e89444727733a2    0.002194
Jnursssmith5263                       0.036932
jpadilla                              0.012773
not-logged-in-0b21b25d20a3f6ed12d8    0.002833
RaeneVigil                            0.000599
DevionJ                               0.006673
Jonas_Cross                           0.000273
not-logged-in-8cf2e6929490167ca6ab    0.006933
framosuw                              0.000381
ComeMyChild                           0.000495
shh-imhuntingscience                  0.001319
not-logged-in-9b697eb9a6436669344b    0.000774
not-logged-in-9a45911eb71184fe7c7f    0.000825
AdamMorse                             0.000253
leclercsimon                          0.001337
peakscience7                          0.027333
41Letters                             0.000696
acapirala                             0.023996
aoliveira2020                         0.037457
Lavadude                              0.046959
not-logged-in-4ea23f5fffc8b98635af    0.002217
bookert      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())